###  Created by Luis Alejandro (alejand@umich.edu)

In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
import tensorflow as tf
from tfrecord_dataset import feature_description
from aircraft_models import AirBinaryRNN

In [2]:
# Constants
AUTOTUNE = tf.data.experimental.AUTOTUNE
MFCC_SIZE = 128
BUFFER_SIZE = 1000
BATCH_SIZE = 64
WINDOW_SIZE = 50

In [3]:
# Dataset location
train_file = 'exports/2020-02-07 01-09-35/train.tfrecord'
test_file = 'exports/2020-02-07 01-09-35/test.tfrecord'

In [4]:
# Parses observation from proto format
def tf_parse_observation(example):
    observation = tf.io.parse_single_example(example, feature_description)
    mfcc = observation['mfcc']
    samples = observation['samples']
    observation['spec'] = tf.reshape(observation['spec'],(mfcc,samples))
    return observation
    
# Converts into correct format for training (input,output) = (spec,label)
def tf_transform_observation(data):
    return tf.expand_dims(data['spec'],-1), data['label'] == b'Airbus'

def tf_make_sequence(spec, label):
    samples = tf.shape(spec)[1]
    start = tf.range(0,samples - WINDOW_SIZE, int(WINDOW_SIZE/2))
    end = tf.range(WINDOW_SIZE, samples, int(WINDOW_SIZE/2))
    sequence = tf.map_fn(lambda index: spec[:,index[0]:index[1]],
                         tf.stack([start, end], axis=1),
                         back_prop=False,
                         dtype=tf.float32)
    return sequence,label

In [5]:
# Creates training data pipeline
train_dataset = tf.data.TFRecordDataset([train_file])
train_dataset = train_dataset.map(tf_parse_observation,num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(tf_transform_observation,num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(tf_make_sequence, num_parallel_calls=AUTOTUNE).cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

In [6]:
# Creates test data pipeline
test_dataset = tf.data.TFRecordDataset([test_file])
test_dataset = test_dataset.map(tf_parse_observation,num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(tf_transform_observation,num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(tf_make_sequence, num_parallel_calls=AUTOTUNE).cache()
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

In [7]:
# Creates model
builder = AirBinaryRNN()
model = builder.build_model(input_shape=[None, MFCC_SIZE, WINDOW_SIZE, 1])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 128, 50, 1) 0         
_________________________________________________________________
Conv1 (TimeDistributed)      (None, None, 124, 46, 32) 832       
_________________________________________________________________
Pool1 (TimeDistributed)      (None, None, 41, 15, 32)  0         
_________________________________________________________________
Conv2 (TimeDistributed)      (None, None, 37, 11, 32)  25632     
_________________________________________________________________
Pool2 (TimeDistributed)      (None, None, 12, 3, 32)   0         
_________________________________________________________________
Flatten (TimeDistributed)    (None, None, 1152)        0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                151680

In [8]:
# Training model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
model.fit(train_dataset,epochs=5,validation_data=test_dataset)

Epoch 1/5
14/14 [==============================] - 8s 606ms/step - loss: 0.1162 - accuracy: 0.9772 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
14/14 [==============================] - 2s 121ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 3/5
14/14 [==============================] - 2s 121ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 4/5
14/14 [==============================] - 2s 121ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 5/5
14/14 [==============================] - 2s 123ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
